In [92]:
import pandas as pd
import numpy as np
import pyodbc
from datetime import timedelta
from tqdm import tqdm
from scipy.stats import pearsonr

In [93]:
name_db = 'StocksPriceIBrX'

data_conection = ("Driver={SQL Server};"
                 "Server=DESKTOP-0EKP870;"
                 f"Database={name_db};")
cnxn = pyodbc.connect(data_conection)
cursor = cnxn.cursor()

In [94]:
data_frame = pd.read_excel(r'data/StocksFrameIBrX.xlsx')
data_frame = data_frame.set_index('RptDt')
display(data_frame.head(3))

,TckrLst,ClsPrc
RptDt,,
1996-01-02,"['BES 4', 'CES 3', 'CES 4', 'CHA 4', 'CSI 6', ...","[5.04, 23.0, 26.7, 0.16, 1.28, 46.0, 2.5, 10.7..."
1996-01-03,"['BES 4', 'CES 3', 'CES 4', 'CHA 4', 'CSI 6', ...","[5.35, 24.0, 27.5, 0.17, 1.37, 54.0, 0.7, 10.7..."
1996-01-04,"['BES 4', 'CES 3', 'CES 4', 'CHA 4', 'CSI 6', ...","[5.5, 22.0, 28.49, 0.17, 1.35, 153.5, 2.5, 7.2..."


In [95]:
new_tickers = []
for i in tqdm(range(len(data_frame)), position=0, leave=True):
    df_to_list = data_frame.iloc[i][0].split(',')
    df_to_list[0] = df_to_list[0].replace('[',' ')
    df_to_list[-1] = df_to_list[-1].replace(']','')
    for j, element in enumerate(df_to_list):
        df_to_list[j] = element.replace(' ', '')
        df_to_list[j] = df_to_list[j][1:len(df_to_list[j])-1]
        if len(df_to_list[j]) >=5:
            aux_begin = df_to_list[j][:3]
            aux_end = df_to_list[j][4:]
            df_to_list[j] = aux_begin + aux_end
    new_tickers.append(df_to_list)

100%|████████████████████████████████████████████████████████████████████████████| 9495/9495 [00:01<00:00, 5387.99it/s]


In [96]:
new_prices = []
for i in tqdm(range(len(data_frame)), position=0, leave=True):
    df_to_list = data_frame.iloc[i][1].split(',')
    df_to_list[0] = df_to_list[0].replace('[',' ')
    df_to_list[-1] = df_to_list[-1].replace(']','')
    for j, element in enumerate(df_to_list):
        df_to_list[j] = element.replace(' ', '')
        df_to_list[j] = float(df_to_list[j])
    new_prices.append(df_to_list)

100%|████████████████████████████████████████████████████████████████████████████| 9495/9495 [00:01<00:00, 7323.42it/s]


In [97]:
data_frame.drop(['TckrLst', 'ClsPrc'], axis=1)
data_frame['TckrLst'] = new_tickers
data_frame['ClsPrc'] = new_prices
display(data_frame.head(3))

,TckrLst,ClsPrc
RptDt,,
1996-01-02,"[BES4, CES3, CES4, CHA4, CSI6, ELP6, FTS4, LEV...","[5.04, 23.0, 26.7, 0.16, 1.28, 46.0, 2.5, 10.7..."
1996-01-03,"[BES4, CES3, CES4, CHA4, CSI6, ELP6, FTS4, LEV...","[5.35, 24.0, 27.5, 0.17, 1.37, 54.0, 0.7, 10.7..."
1996-01-04,"[BES4, CES3, CES4, CHA4, CSI6, ELP6, FTS4, LEV...","[5.5, 22.0, 28.49, 0.17, 1.35, 153.5, 2.5, 7.2..."


In [105]:
begin_date = pd.to_datetime('2010-12-28')
#end_date = pd.to_datetime('2010-12-27')
end_date = data_frame.index[-1]
print(f'Begin Date: {begin_date}')
print(f'End Date: {end_date}')

Begin Date: 2010-12-28 00:00:00
End Date: 2021-12-30 00:00:00


In [106]:
window = 365

In [107]:
#Capta os preços de acordo com os índices
def prices_stocks (data_frame, index_stock, start, stop):
    prices = []
    count = 0
    for k in range(data_frame.index.get_loc(start), data_frame.index.get_loc(stop)+1):
        prices.append(data_frame.iloc[k][1][index_stock[count]])
        count = count + 1
    return prices

In [108]:
#Estima as posições de cada ação nas listas de tickres no período de 1 ano
def prices_to_estimation (data_frame, intersection, start, stop, stock):
    index_stock = []
    for j in range(data_frame.index.get_loc(start), data_frame.index.get_loc(stop)+1):
        index_stock.append(data_frame.iloc[j][0].index(stock))
    prices = prices_stocks(data_frame, index_stock, start, stop)
    return prices

In [109]:
#Encontra a interceção entre duas listas
def common_elements (current_tickrs, previous_tickrs):
    intersection = []
    aux = []
    for element in current_tickrs:
        validation = [element in previous_tickrs[o] for o in range(len(previous_tickrs))]
        if all(validation):
            intersection.append(element)

    return intersection

In [110]:
def create_table (stocks_dataframe, id_table, name_db):
    columnName = list(stocks_dataframe.columns)
    columnDataType = ['FLOAT' for i in range (len(columnName))]
    
    #Código em sql para criar as colunas
    createTableStatement = f'USE {name_db} \n CREATE TABLE Table{id_table} ('
    for i in range(len(columnDataType)):
        createTableStatement = createTableStatement + '\n' + columnName[i] + ' ' + columnDataType[i] + ','
    createTableStatement = createTableStatement[:-1] + ' );'

    #Código em sql para inserir valores nas colunas
    InsertValuesTable = f'INSERT INTO Table{id_table}('

    for i in range(len(columnName)):
        InsertValuesTable = InsertValuesTable + columnName[i] + ',' + ' '
    InsertValuesTable = InsertValuesTable[:-2] + ')\n VALUES'
    subset = stocks_dataframe[list(stocks_dataframe.columns)]
    tuples = [tuple(x) for x in subset.to_numpy()]
    values_rows = '('
    for numeric in tuples[0]:
        values_rows = values_rows + str(numeric) + ',' + ' '
    values_rows = values_rows[:-2] + ')' + ',' + '\n'
    for i in range(1, len(tuples)-1):
        values_rows = values_rows + '('
        for numeric in tuples[i]:
            values_rows = values_rows + str(numeric) + ',' + ' '
        values_rows = values_rows[:-2] + ')' + ',' + '\n'
    values_rows = values_rows + '('
    for numeric in tuples[-1]:
        values_rows = values_rows + str(numeric) + ',' + ' '
    values_rows = values_rows[:-2] + ')' + '\n'
    
    cursor.execute(createTableStatement + InsertValuesTable + values_rows)
    cnxn.commit()

In [111]:
for i in tqdm(range(data_frame.index.get_loc(begin_date), data_frame.index.get_loc(end_date)+1),\
              position=0, leave=True, desc = 'elementos comuns'):
    start = data_frame.index[i] -  timedelta(days=window)
    stop = data_frame.index[i]
    
    current_tickrs = data_frame.loc[stop]['TckrLst']
    previous_tickrs = data_frame.loc[start:stop]['TckrLst'].tolist()
    
    intersection = common_elements(current_tickrs, previous_tickrs)
    intersection = list(set(intersection))
    if len(intersection) !=0:
        all_prices = []
        for stock in intersection:
            prices = prices_to_estimation (data_frame, intersection, start, stop, stock)
            all_prices.append(prices)

        stocks_dataframe = pd.DataFrame(np.array(all_prices).T, columns=intersection)
        create_table(stocks_dataframe, i, name_db)



elementos comuns: 100%|██████████████████████████████████████████████████████████| 4021/4021 [5:57:51<00:00,  5.34s/it]
